In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lgr').getOrCreate()

24/05/08 09:26:05 WARN Utils: Your hostname, willians-desktop resolves to a loopback address: 127.0.1.1; using 192.168.1.73 instead (on interface enp1s0)
24/05/08 09:26:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/08 09:26:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.format('libsvm').load('sample_libsvm_data.txt')

24/05/08 09:27:49 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [3]:
df.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [11]:
train, test = df.randomSplit([0.7, 0.3])

In [5]:
from pyspark.ml.classification import LogisticRegression

In [6]:
lrg = LogisticRegression()

In [12]:
trained_lrg = lrg.fit(train)

24/05/08 09:45:54 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [13]:
result = trained_lrg.transform(test)

In [14]:
result.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[32.3635635244885...|[0.99999999999999...|       0.0|
|  0.0|(692,[123,124,125...|[44.8795962422738...|           [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[40.4210228146256...|           [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[23.9053228127264...|[0.99999999995849...|       0.0|
|  0.0|(692,[124,125,126...|[29.8144290071048...|[0.99999999999988...|       0.0|
|  0.0|(692,[126,127,128...|[37.6402738787961...|           [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|[13.1082637283471...|[0.99999797160507...|       0.0|
|  0.0|(692,[126,127,128...|[20.6208607729692...|[0.99999999889216...|       0.0|
|  0.0|(692,[126,127,128...|[31.0644161621325...|[0.99999999999996...|       0.0|
|  0.0|(692,[127

In [16]:
test_2 = test.select(['features'])
result = trained_lrg.transform(test_2)
result.show()

+--------------------+--------------------+--------------------+----------+
|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|(692,[95,96,97,12...|[32.3635635244885...|[0.99999999999999...|       0.0|
|(692,[123,124,125...|[44.8795962422738...|           [1.0,0.0]|       0.0|
|(692,[124,125,126...|[40.4210228146256...|           [1.0,0.0]|       0.0|
|(692,[124,125,126...|[23.9053228127264...|[0.99999999995849...|       0.0|
|(692,[124,125,126...|[29.8144290071048...|[0.99999999999988...|       0.0|
|(692,[126,127,128...|[37.6402738787961...|           [1.0,0.0]|       0.0|
|(692,[126,127,128...|[13.1082637283471...|[0.99999797160507...|       0.0|
|(692,[126,127,128...|[20.6208607729692...|[0.99999999889216...|       0.0|
|(692,[126,127,128...|[31.0644161621325...|[0.99999999999996...|       0.0|
|(692,[127,128,129...|[32.5833332929749...|[0.99999999999999...|       0.0|
|(692,[150,1

In [18]:
results = trained_lrg.evaluate(test)

In [35]:
results.accuracy

1.0

In [28]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [36]:
evaluator = BinaryClassificationEvaluator()
evaluation = evaluator.evaluate(results.predictions)